In [1]:
import torch
import math
import torch.nn as nn

In [2]:
message_size = 500
hidden_node_features = 100

class SummationMpnn(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_node_features = 500 #size from constant file
        self.edge_features = 4
        self.message_size = 100
        self.message_passes = 3
    def forward(self,nodes:tuple,edges:torch.tensor)->None:
        adjacency = torch.sum(edges, dim=3)
        print("adj is ",adjacency.shape)
        (edge_batch_batch_idc,
        edge_batch_node_idc,
        edge_batch_nghb_idc) = adjacency.nonzero(as_tuple=True)
        # print("edges various ares",(edge_batch_batch_idc.shape,
        # edge_batch_node_idc.shape,
        # edge_batch_nghb_idc.shape))
        #means picking out non zero waala indiced..gives connecting nodes
        (node_batch_batch_idc,
        node_batch_node_idc) = adjacency.sum(-1).nonzero(as_tuple=True)
        print("nodes various ares",(node_batch_batch_idc,node_batch_node_idc))

        same_batch = node_batch_batch_idc.view(-1,1)== edge_batch_batch_idc
        same_node  = node_batch_node_idc.view(-1, 1) == edge_batch_node_idc
        print("same batch,node",same_batch,   same_node)
        message_summation_matrix = (same_batch * same_node).float()
        print("message ",message_summation_matrix)
        edge_batch_edges = edges[edge_batch_batch_idc, edge_batch_node_idc, edge_batch_nghb_idc, :]
        print("edge_batch_edges ",edge_batch_edges.shape,'hello ',edges.shape)

        hidden_nodes = torch.zeros(nodes.shape[0],
                                   nodes.shape[1],
                                   self.hidden_node_features,
                                   device='cuda')
        hidden_nodes[:nodes.shape[0], :nodes.shape[1], :nodes.shape[2]] = nodes.clone()#padding upto 13 nodes,with features to 100....we have only 9 features here
        node_batch_nodes = hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :]#picking out the same batch

        print("yoyo ",node_batch_nodes.shape)
        print("yoyo2",hidden_nodes.shape)

        for _ in range(self.message_passes):
            edge_batch_nodes = hidden_nodes[edge_batch_batch_idc, edge_batch_node_idc, :]#getting hi  13*13*100

            edge_batch_nghbs = hidden_nodes[edge_batch_batch_idc, edge_batch_nghb_idc, :]#getting neighs(hj)  13*13*1#why its 1
            print("in message pass ",edge_batch_nghbs.shape, "nodes ",edge_batch_nodes.shape)

            message_terms    = self.message_terms(edge_batch_nodes,  
                                                  edge_batch_nghbs,
                                                  edge_batch_edges)

            if len(message_terms.size()) == 1:  # if a single graph in batch
                message_terms = message_terms.unsqueeze(0)

            # the summation in eq. 1 of the NMPQC paper happens here
            messages = torch.matmul(message_summation_matrix, message_terms)

            node_batch_nodes = self.update(node_batch_nodes, messages)
            hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :] = node_batch_nodes.clone()

        node_mask = adjacency.sum(-1) != 0
        output    = self.readout(hidden_nodes, nodes, node_mask)
        return hidden_nodes
        #return output


In [3]:
class Attns2v(SummationMpnn):
    """
    The "message neural network" model.
    """
    def __init__(self):
        super().__init__()
        #self.constants       = constants
        message_weights      = torch.Tensor(message_size,
                                            hidden_node_features,
                                           4)
        if True:#== "cuda":
            message_weights = message_weights.to("cuda", non_blocking=True)

        self.message_weights = torch.nn.Parameter(message_weights)

        self.gru             = torch.nn.GRUCell(
            input_size=message_size,
            hidden_size = hidden_node_features,
            bias=True
        )
        #4 edge features
        self.mlp1 = MLP(
            in_features=100,
            hidden_layer_sizes=[250] * 1,
            out_features=100,
            dropout_p=0.2
        )
        self.mlp2 = MLP(4,[250]*4,4,dropout_p=0)

        self.reset_parameters()

    def reset_parameters(self) -> None:
        stdev = 1.0 / math.sqrt(self.message_weights.size(1))
        self.message_weights.data.uniform_(-stdev, stdev)

    def message_terms(self, nodes,node_neighbours,edges):
        edges_v = edges.view(-1,1,1)#it gives value for edges.....type of edge..size->(6,4)->in molecule with 6 edges kind
        neighs = edges_v*node_neighbours.view(-1,1,1)#multiplying each by this number
        m1,b1 = [],[]
        for i in range(4):
            m1.append(edges[:,i,:]*self.mlp1[i](neighs[:,i,:]))
            b1.append(edges[:,i,:]*self.mlp2[i](neighs[:,i,:]))
        m1 = sum(m1)
        b1 = sum(b1)
        a = self.Softmax(b1)#check this Neigh(j) not understabbale
        

        return output

    def update(self, nodes, messages):
        return self.gru(messages, nodes)
    
    def readout(self,nodes_out,nodes_in):#two different readout will be here.....1st and 2nf is adp readout
        
        #mlp is single linear layer....signle hidden neural net
        
        p = self.emb(torch.cat[nodes_out,nodes_in])
        






SyntaxError: invalid syntax (Temp/ipykernel_11828/498239923.py, line 41)

In [ ]:
: